In [1]:
import os
import shutil
import pickle
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt

from skopt.space import Real, Categorical, Integer

from bcnf.gp_minimize.gp_minimize import gp_minimize_fixed
from bcnf.simulation.physics import get_data
from bcnf.models.cnf import CondRealNVP
from bcnf.models.feature_network import FullyConnectedFeatureNetwork
from bcnf.eval.crossvalidate import cross_validate
from bcnf.errors import TrainingDivergedError
from bcnf.utils import get_dir

from bcnf.simulation.sampling import generate_data

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Setup

In [2]:
# HACK: scikit-optimize is not maintained anymore and this is a quick fix to make it work
# https://github.com/scikit-optimize/scikit-optimize/issues/1171#:~:text=To%20avoid%20this%20error%20in%20existing%20code%2C%20use%20int%20by%20itself.
np.int = np.int64

In [3]:
# Store and load the progress in the Huggingface repository
checkpoint_file = os.path.join(get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", create=True), 'checkpoint_improved.pkl')
metrics_dir = get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", "metrics", create=True)

print(os.path.abspath(checkpoint_file))
print(os.path.abspath(metrics_dir))

/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/checkpoint_improved.pkl
/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/metrics


## Data

In [4]:
param_names = ['x0_x','x0_y','x0_z','v0_x','v0_y','v0_z','g','w_x','w_y','w_z','b','A','Cd','rho','m','a_x','a_y','a_z','r']

In [5]:
# dataset_name = "hyperparameter_optimization_trajectories"

# if not os.path.exists(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl')):
#     data = generate_data(
#         n=2000,
#         type="trajectory",
#         SPF=1/30,
#         T=3,
#         config_file=os.path.join(get_dir("configs"), "throw_upwards.yaml"),
#         verbose=True,
#         break_on_impact=False,
#         name=dataset_name
#     )
# else:
#     with open(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl'), 'rb') as f:
#         data = pickle.load(f)

In [6]:
# X = np.array(data['traj'])
# y = np.column_stack([np.array(data[param]) for param in param_names])


# X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
# y_tensor = torch.Tensor(y)

# X_tensor.shape, y_tensor.shape

## Hyperparameter

In [7]:
X, y = get_data(
    T=1.0,
    dt=1 / 30,
    N=2_000,
    break_on_impact=False
)

X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
y_tensor = torch.Tensor(y)

X_tensor.shape, y_tensor.shape

100%|██████████| 2000/2000 [00:02<00:00, 938.26it/s]


(torch.Size([2000, 90]), torch.Size([2000, 17]))

In [8]:
model_size = y_tensor.shape[1]
feature_size = X_tensor.shape[1]

print(f"Model size: {model_size}")
print(f"Feature size: {feature_size}")

Model size: 17
Feature size: 90


In [9]:
# These parameters have worked well in pilot experiments
optimizer_kwargs = {
    "lr": 2e-4
}

lr_scheduler_kwargs = {
    "mode": "min",
    "factor": 0.5,
    "patience": 250,
    "threshold_mode": "abs",
    "threshold": 1e-1,
}

In [10]:
# Define the search space
search_spaces = {
    'condition_size': Integer(1, 2048),
    'model_nested_size': Integer(16, 1024),
    'model_nested_layers': Integer(1, 8),
    'model_n_blocks': Integer(4, 32),
    'model_act_norm': Categorical([True, False]),
    'model_dropout': Real(0.0, 0.5),
    'feature_network_hidden_size': Integer(16, 256),
    'feature_network_hidden_layers': Integer(0, 16),
    'feature_network_dropout': Real(0.0, 0.5),
}

## Helper Functions

In [11]:
def param_index(name: str, search_spaces: dict[str, Real | Integer | Categorical]):
    """
    Get the index of a parameter in the search space to match the order of the parameters in the optimization function.
    
    Parameters
    ----------
    name : str
        The name of the parameter.
    search_spaces : dict[str, Real | Integer | Categorical]
        The search space.

    Returns
    -------
    int
        The index of the parameter in the search space.
    """
    return list(search_spaces.keys()).index(name)

In [12]:
def score_parameters(params: list):
    print({k: v for k, v in zip(search_spaces.keys(), params)})

    # Catch training errors (such as divergence) to filter out bad parameter sets and speed up the optimization
    try:
        # Use cross-validation to estimate the performance of the model
        fold_metrics = cross_validate(
            model_class=CondRealNVP,
            model_kwargs={
                "size": model_size,
                "nested_sizes": [params[param_index('model_nested_size', search_spaces)]] * params[param_index('model_nested_layers', search_spaces)],
                "n_blocks": params[param_index('model_n_blocks', search_spaces)],
                "n_conditions": params[param_index('condition_size', search_spaces)],
                "act_norm": params[param_index('model_act_norm', search_spaces)],
                "dropout": params[param_index('model_dropout', search_spaces)],
            },
            feature_network_class=FullyConnectedFeatureNetwork,
            feature_network_kwargs={
                "sizes": [feature_size]
                    + [params[param_index('feature_network_hidden_size', search_spaces)]] * params[param_index('feature_network_hidden_layers', search_spaces)]
                    + [params[param_index('condition_size', search_spaces)]],
                "dropout": params[param_index('feature_network_dropout', search_spaces)],    
            },
            optimizer_class=torch.optim.Adam,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            X=X_tensor,
            y=y_tensor,
            n_epochs=50_000,
            val_loss_patience=500,
            val_loss_tolerance=1e-1,  # Improvements to treat as significant
            val_loss_tolerance_mode="abs",
            timeout=60 * 60,  # 1 hour
            batch_size=256,
            device=device,
            verbose=True,  # Print the progress
            n_splits=3,
            errors="raise"  # Raise errors to stop the optimization
        )

        # Save the loss_history and metrics for analysis
        with open(os.path.join(metrics_dir, f'params_{"_".join([str(p) for p in params])}.pkl'), 'wb') as f:
            pickle.dump(fold_metrics, f)

    except TrainingDivergedError as e:
        print(e)
        return 100  # A big number (bad score) to avoid this parameter set

    # Print the average validation loss and its standard deviation during optimization
    val_loss_list = [r['val_loss'][1] for r in fold_metrics]  # each val_loss value is a tuple (epoch, loss)
    print(f'Val Loss: {np.mean(val_loss_list):.4f} ± {np.std(val_loss_list):.4f}')

    # Return the upper confidence bound of the validation loss
    # This encourages the optimization to find good AND reliable parameter sets
    return np.mean(val_loss_list) + np.std(val_loss_list)

In [13]:
def save_checkpoint(result):
    """
    Save the result of the optimization to a checkpoint file.
    Used as a callback in the optimization function.
    
    Parameters
    ----------
    result : OptimizeResult
        The result of the optimization.
    """
    # Safely write the result to a temporary file first without overwriting the checkpoint file
    with open(checkpoint_file + ".tmp", 'wb') as f:
        # Ignore
        # - result['specs']['args']['func']
        # - result['specs']['args']['callback']
        # because it causes problems when reading somewhere else
        result_no_func = copy.deepcopy(result)
        del result_no_func['specs']['args']['func']
        del result_no_func['specs']['args']['callback']
        pickle.dump(result_no_func, f)

    # Delete the old checkpoint file and rename the temporary file
    shutil.move(checkpoint_file + ".tmp", checkpoint_file)

## Optimization

In [14]:
# Numer of random initial points to explore the search space
N_STEPS_INIT = 30

# Number of iterations to run the optimization in total
N_STEPS = 100

In [15]:
# Load the checkpoint if it exists
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')

    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)

        # Re-assign the function and callback because they are not picklable
        checkpoint['specs']['args']['func'] = score_parameters
        checkpoint['specs']['args']['callback'] = save_checkpoint

    print(f'Resuming from iteration {len(checkpoint.x_iters)}')

    n_initial_points = max(0, N_STEPS_INIT - len(checkpoint.x_iters))
    n_calls_remaining = max(0, N_STEPS - len(checkpoint.x_iters))
    x0 = checkpoint.x_iters
    y0 = checkpoint.func_vals
else:
    print('No checkpoint found. Starting new optimization')
    checkpoint = None

    n_initial_points = N_STEPS_INIT
    n_calls_remaining = N_STEPS

    x0 = None
    y0 = None
    
print(f'Running with {n_initial_points} initial points and {n_calls_remaining} remaining iterations')

Loading checkpoint from /home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/checkpoint_improved.pkl
Resuming from iteration 60
Running with 0 initial points and 40 remaining iterations


In [16]:
# You might want to adjust the n_calls or other parameters based on the checkpoint
result = gp_minimize_fixed(
    func=score_parameters,
    dimensions=search_spaces.values(),
    n_initial_points=n_initial_points,  # Number of random points before starting the optimization
    n_calls=n_calls_remaining,  # Number of iterations
    random_state=2024_03_25,
    verbose=True,
    acq_func="EI",  # Expected Improvement https://arxiv.org/pdf/1009.5419.pdf
    initial_point_generator="halton", # https://en.wikipedia.org/wiki/Halton_sequence
    callback=save_checkpoint,
    x0=x0,
    y0=y0)

0 initial points will be randomly generated
Iteration No: 1 started. Searching for the next optimal point.
Telling optimizer about 60 initial points
Iteration No: 1 ended. Search finished for the next optimal point.
Time taken: 0.9767
Function value obtained: -18.6476
Current minimum: -57.6940
Iteration No: 2 started. Searching for the next optimal point.
Asking optimizer for next point 1: [992, 25, 8, 27, True, 0.0, 87, 7, 0.3220522680512488]
{'condition_size': 992, 'model_nested_size': 25, 'model_nested_layers': 8, 'model_n_blocks': 27, 'model_act_norm': True, 'model_dropout': 0.0, 'feature_network_hidden_size': 87, 'feature_network_hidden_layers': 7, 'feature_network_dropout': 0.3220522680512488}


Train: -47.0810 - Val: -48.2658 (avg: -48.2620, min: -48.1994) | lr: 3.91e-07 - Patience: 500/500:  23%|██▎       | 11739/50000 [51:55<2:49:13,  3.77it/s]
Train: -44.6245 - Val: -45.4706 (avg: -45.4670, min: -45.3916) | lr: 3.91e-07 - Patience: 500/500:  25%|██▍       | 12426/50000 [54:13<2:43:56,  3.82it/s]
Train: -45.9818 - Val: -46.6818 (avg: -46.6778, min: -46.6211) | lr: 3.91e-07 - Patience: 500/500:  26%|██▌       | 13015/50000 [58:15<2:45:32,  3.72it/s]


Val Loss: -46.7953 ± 1.1835
Iteration No: 2 ended. Search finished for the next optimal point.
Time taken: 9864.8863
Function value obtained: -45.6119
Current minimum: -57.6940
Iteration No: 3 started. Searching for the next optimal point.
Asking optimizer for next point 2: [836, 191, 7, 21, True, 0.0, 159, 5, 0.37867605407596466]
{'condition_size': 836, 'model_nested_size': 191, 'model_nested_layers': 7, 'model_n_blocks': 21, 'model_act_norm': True, 'model_dropout': 0.0, 'feature_network_hidden_size': 159, 'feature_network_hidden_layers': 5, 'feature_network_dropout': 0.37867605407596466}


Train: -46.1406 - Val: -39.7691 (avg: -39.7616, min: -39.6890) | lr: 7.81e-07 - Patience: 500/500:  15%|█▍        | 7265/50000 [23:38<2:19:05,  5.12it/s]
Train: -46.5826 - Val: -35.9967 (avg: -36.0123, min: -35.9978) | lr: 1.56e-06 - Patience: 500/500:  11%|█         | 5555/50000 [18:06<2:24:56,  5.11it/s]
Train: -44.7416 - Val: -34.1657 (avg: -34.2247, min: -34.3619) | lr: 1.25e-05 - Patience: 500/500:  10%|▉         | 4990/50000 [16:15<2:26:42,  5.11it/s]


Val Loss: -36.9045 ± 3.2040
Iteration No: 3 ended. Search finished for the next optimal point.
Time taken: 3482.3498
Function value obtained: -33.7005
Current minimum: -57.6940
Iteration No: 4 started. Searching for the next optimal point.
Asking optimizer for next point 3: [236, 168, 7, 19, True, 0.0, 76, 7, 0.2690183822722158]
{'condition_size': 236, 'model_nested_size': 168, 'model_nested_layers': 7, 'model_n_blocks': 19, 'model_act_norm': True, 'model_dropout': 0.0, 'feature_network_hidden_size': 76, 'feature_network_hidden_layers': 7, 'feature_network_dropout': 0.2690183822722158}


...

Train: 4.3147 - Val: 1237.8316 (avg: 30.4067, min: -33.7473) | lr: 2.00e-04 - Patience: 42/500:   9%|▉         | 4464/50000 [14:03<2:23:21,  5.29it/s]   


Error in fold 0: Loss exploded to 155130.265625 at epoch 4464.166666666667
Loss exploded to 155130.265625 at epoch 4464.166666666667
Iteration No: 8 ended. Search finished for the next optimal point.
Time taken: 844.9519
Function value obtained: 100.0000
Current minimum: -57.6940
Iteration No: 9 started. Searching for the next optimal point.
Asking optimizer for next point 8: [288, 16, 2, 23, True, 0.07146908809713125, 114, 9, 0.3589314349977031]
{'condition_size': 288, 'model_nested_size': 16, 'model_nested_layers': 2, 'model_n_blocks': 23, 'model_act_norm': True, 'model_dropout': 0.07146908809713125, 'feature_network_hidden_size': 114, 'feature_network_hidden_layers': 9, 'feature_network_dropout': 0.3589314349977031}


Train: -47.8907 - Val: -49.6865 (avg: -49.6816, min: -49.6296) | lr: 7.81e-07 - Patience: 500/500:  23%|██▎       | 11650/50000 [28:25<1:33:33,  6.83it/s]
Train: -47.7180 - Val: -49.7188 (avg: -49.6879, min: -49.6388) | lr: 7.81e-07 - Patience: 500/500:  26%|██▌       | 12968/50000 [32:09<1:31:50,  6.72it/s]
Train: -6.7625 - Val: -7.1735 (avg: -7.2033, min: -7.2048) | lr: 2.00e-04 - Patience: 1/500:   1%|          | 262/50000 [00:38<2:02:42,  6.76it/s]  


Error in fold 2: Loss exploded to 684353.375 at epoch 262.5
Loss exploded to 684353.375 at epoch 262.5
Iteration No: 9 ended. Search finished for the next optimal point.
Time taken: 3675.5881
Function value obtained: 100.0000
Current minimum: -57.6940
Iteration No: 10 started. Searching for the next optimal point.
Asking optimizer for next point 9: [1241, 129, 3, 32, True, 0.2615379899589816, 67, 7, 0.1750433289775834]
{'condition_size': 1241, 'model_nested_size': 129, 'model_nested_layers': 3, 'model_n_blocks': 32, 'model_act_norm': True, 'model_dropout': 0.2615379899589816, 'feature_network_hidden_size': 67, 'feature_network_hidden_layers': 7, 'feature_network_dropout': 0.1750433289775834}


Train: -47.7770 - Val: -50.7355 (avg: -50.7271, min: -50.7191) | lr: 7.81e-07 - Patience: 185/500:  21%|██        | 10340/50000 [40:16<2:38:23,  4.17it/s]